# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

When cleaning the Price variable, the first choice I made was to remove any "$" and ","s that might be in the column due to seeing that the column type intially was object. From there, I made sure to coerce the column into a more appropriate type like float. With all this done, I see there's no missing values, as all the values were able to be properly coerced.

2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 

There are 9848 entries missing out of 11294. This is a concern, as this means that 87% of the entries are missing. When cross-tabulating, we see that the majority of the missing entries when the force type was "bodily force".

3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.

4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

After seeing most of most of the entries were " ", I replaced those with NAs to better reflect the data. From there, by cross-tabulating with sentence type, I saw all the NAs were associated with sentence types 4 and 9, which might be for cases where there's no sentence being handed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

airbnb = pd.read_csv('./data/airbnb_hw.csv',low_memory=False)

var = 'Price' 
print('Before coercion: \n', airbnb[var].describe(),'\n')
airbnb[var].hist(bins=50) 

airbnb["Price"] = airbnb["Price"].str.replace(r"[\$,]", "", regex=True).astype(float)

airbnb['Price'] = pd.to_numeric(airbnb['Price'], errors='coerce')

airbnb['price_nan'] = airbnb['Price'].isnull() 

print('After coercion: \n', airbnb['Price'].describe(),'\n') 
airbnb['Price'].hist(bins = 50) 
print('Total Missings: \n', sum(airbnb['price_nan']),'\n')

In [ ]:
police = pd.read_csv('./data/mn_police_use_of_force.csv',low_memory=False)

var = "subject_injury"

police["injury_na"] = police[var].isnull()

print(police["injury_na"].value_counts(), '\n')

print(police["subject_injury"].value_counts(), '\n')

print(police[var].unique(), '\n')

#police[var] = police[var].replace(nan ,np.nan)
#police[var].value_counts()

pd.crosstab(police["injury_na"], police["force_type"])

In [ ]:
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
justice = pd.read_csv(url,low_memory=False)

print(justice["WhetherDefendantWasReleasedPretrial"].unique(), '\n')

justice["WhetherDefendantWasReleasedPretrial"] = justice["WhetherDefendantWasReleasedPretrial"].replace(9, np.nan)

print(justice["WhetherDefendantWasReleasedPretrial"].value_counts(), '\n')

In [ ]:
print(justice['ImposedSentenceAllChargeInContactEvent'].value_counts(dropna=False).head(20))

justice['ImposedSentenceAllChargeInContactEvent'] = justice['ImposedSentenceAllChargeInContactEvent'].replace(" ", np.nan)

pd.crosstab(justice['SentenceTypeAllChargesAtConvictionInContactEvent'],justice['ImposedSentenceAllChargeInContactEvent'].isna(), margins=True)

In [ ]:
print(justice['ImposedSentenceAllChargeInContactEvent'].value_counts(dropna=False).head(20))

justice['ImposedSentenceAllChargeInContactEvent'] = justice['ImposedSentenceAllChargeInContactEvent'].replace(" ", np.nan)

pd.crosstab(justice['SentenceTypeAllChargesAtConvictionInContactEvent'],justice['ImposedSentenceAllChargeInContactEvent'].isna(), margins=True)


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?

The original range of years is from 1900 - 2026. Since the 1940s, attacks have constantly fluctuated over time, but are around the same levels they were at at 1940.

4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?

There were 4333 men out of 5053 victims.

6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?

The proportion of unprokved attacks are 74.4%.

7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?

Sharks are far more likely to launch unprovoked attacks on men than women, attacks are much more likely to be fatal when unprovoked, and fatality rates are slightly higher for men than for women.

9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

About 12.87% of attacks are by white sharks

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd  

shark_url = 'https://sharkattackfile.net/spreadsheets/GSAF5.xls'
shark = pd.read_excel(shark_url)

print(shark.shape, '\n') 

shark = shark.dropna(axis=1, how='all')
print(shark.shape, '\n')

In [ ]:
shark.loc[shark['Year'] < 1800, 'Year'] = np.nan

print(shark["Year"].describe(), '\n')

shark = shark[shark['Year'] >= 1940]  
shark['Year'].hist(bins=80)

In [ ]:
shark['Age'] = pd.to_numeric(shark['Age'], errors='coerce')
shark['Age'].hist(bins=30)

In [ ]:
print(shark["Sex"].value_counts(dropna=False))

shark["Sex"] = shark["Sex"].str.strip().str.upper()

shark.loc[~shark["Sex"].isin(['M', 'F']), "Sex"] = np.nan

print(shark["Sex"].value_counts(dropna=False))

In [ ]:
shark['Type'] = shark['Type'].str.strip().str.capitalize()
shark['Type'] = shark['Type'].where(shark['Type'].isin(['Unprovoked', 'Provoked']), 'Unknown')
print(shark['Type'].value_counts(normalize=True), '\n')

In [ ]:
shark['Fatal Y/N'] = shark['Fatal Y/N'].str.strip().str.upper()
shark['Fatal Y/N'] = shark['Fatal Y/N'].where(shark['Fatal Y/N'].isin(['Y', 'N']), 'Unknown')
print(shark['Fatal Y/N'].value_counts(normalize=True), '\n')

print(pd.crosstab(shark['Sex'], shark['Type'], normalize='columns'), '\n')

print(pd.crosstab(shark['Type'], shark['Fatal Y/N'], normalize='index'), '\n')

print(pd.crosstab(shark['Sex'], shark['Fatal Y/N'], normalize='index'), '\n')

In [ ]:
shark["Species "] = shark["Species "].astype(str).str.lower()

shark['white_shark'] = shark["Species "].str.contains('white', na=False)
print(shark['white_shark'].mean(), '\n')